In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train = pd.read_csv('../Excel/titanic_train.csv', )
test = pd.read_csv('../Excel/titanic_test.csv')

In [3]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# The name column isn't important at all, I don't think that if you're Michelle you will survive and Michael won't :)
# Same for PassengerId
train.drop(axis=1, columns=['Name', 'PassengerId'], inplace=True)
test.drop(axis=1, columns=['Name', 'PassengerId'], inplace=True)

In [6]:
target = train['Survived']
train.drop(axis=1, columns='Survived', inplace=True)

In [7]:
cat_features = [col for col in train.columns if train[col].dtype == np.dtype('object')]
cat_features

['Sex', 'Ticket', 'Cabin', 'Embarked']

In [8]:
cat = []
for col in cat_features:
    print(f'Col name: {col}, nunique: {train[col].nunique()}')
    if train[col].nunique() < 5:
        cat.append(col)

Col name: Sex, nunique: 2
Col name: Ticket, nunique: 681
Col name: Cabin, nunique: 147
Col name: Embarked, nunique: 3


In [9]:
for col in cat:
    print(f'Col name: {col}, unique: {train[col].unique()}')

Col name: Sex, unique: ['male' 'female']
Col name: Embarked, unique: ['S' 'C' 'Q' nan]


In [10]:
# Will drop Cabin and Ticket
train.drop(axis=1, columns=['Cabin', 'Ticket'], inplace=True)
test.drop(axis=1, columns=['Cabin', 'Ticket'], inplace=True)

In [11]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [12]:
from sklearn.impute import SimpleImputer

Num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

sc = StandardScaler()
ohe = OneHotEncoder(handle_unknown='error', drop='first')
oe = OrdinalEncoder(handle_unknown='error')

In [14]:
num_features =  [col for col in train.columns if train[col].dtype != np.dtype('object')]
num_features

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [15]:
cat_features = [col for col in train.columns if train[col].dtype == np.dtype('object')]
cat_features

['Sex', 'Embarked']

In [16]:
train[num_features] = Num_imputer.fit_transform(train[num_features])
test[num_features] = Num_imputer.transform(test[num_features])

In [17]:
train[cat_features] = cat_imputer.fit_transform(train[cat_features])
test[cat_features] = cat_imputer.transform(test[cat_features])

In [18]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3.0,male,22.0,1.0,0.0,7.2500,S
1,1.0,female,38.0,1.0,0.0,71.2833,C
2,3.0,female,26.0,0.0,0.0,7.9250,S
3,1.0,female,35.0,1.0,0.0,53.1000,S
4,3.0,male,35.0,0.0,0.0,8.0500,S


In [19]:
# We could do it in a single step, but where is the suspense ? it's a very small data with low features so meh
def age_filtering(X):
    if X < 12:
        return 'child'
    elif X in range(12, 18):
        return 'teen'
    else:
        return 'adult'

age_mapping = {
    'child': 0,
    'teen': 1,
    'adult': 2
}    

train['Age_filtering'] = train['Age'].apply(age_filtering)
train['Age_filtering'] = train['Age_filtering'].map(age_mapping)
test['Age_filtering'] = test['Age'].apply(age_filtering)
test['Age_filtering'] = test['Age_filtering'].map(age_mapping)

In [20]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age_filtering
0,3.0,male,22.0,1.0,0.0,7.2500,S,2
1,1.0,female,38.0,1.0,0.0,71.2833,C,2
2,3.0,female,26.0,0.0,0.0,7.9250,S,2
3,1.0,female,35.0,1.0,0.0,53.1000,S,2
4,3.0,male,35.0,0.0,0.0,8.0500,S,2


In [21]:
train[cat_features] = oe.fit_transform(train[cat_features])
test[cat_features] = oe.transform(test[cat_features])

In [22]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age_filtering
0,3.0,1.0,22.0,1.0,0.0,7.2500,2.0,2
1,1.0,0.0,38.0,1.0,0.0,71.2833,0.0,2
2,3.0,0.0,26.0,0.0,0.0,7.9250,2.0,2
3,1.0,0.0,35.0,1.0,0.0,53.1000,2.0,2
4,3.0,1.0,35.0,0.0,0.0,8.0500,2.0,2


In [23]:
encoded_features = [col for col in train.columns if train[col].nunique() < 5]
encoded_features

['Pclass', 'Sex', 'Embarked', 'Age_filtering']

In [30]:
train = pd.get_dummies(train, columns=encoded_features, drop_first=True)
test = pd.get_dummies(test, columns=encoded_features, drop_first=True)

In [31]:
train.head()

,Age,SibSp,Parch,Fare,Pclass_2.0,Pclass_3.0,Sex_1.0,Embarked_1.0,Embarked_2.0,Age_filtering_1,Age_filtering_2
0,22.0,1.0,0.0,7.2500,0,1,1,0,1,0,1
1,38.0,1.0,0.0,71.2833,0,0,0,0,0,0,1
2,26.0,0.0,0.0,7.9250,0,1,0,0,1,0,1
3,35.0,1.0,0.0,53.1000,0,0,0,0,1,0,1
4,35.0,0.0,0.0,8.0500,0,1,1,0,1,0,1


In [36]:
cols = [col for col in train.columns]
cols

['Age',
 'SibSp',
 'Parch',
 'Fare',
 'Pclass_2.0',
 'Pclass_3.0',
 'Sex_1.0',
 'Embarked_1.0',
 'Embarked_2.0',
 'Age_filtering_1',
 'Age_filtering_2']

In [37]:
#Wasn't needed that much
train[cols] = sc.fit_transform(train[cols])
test[cols] = sc.transform(test[cols])

In [38]:
train.head()

,Age,SibSp,Parch,Fare,Pclass_2.0,Pclass_3.0,Sex_1.0,Embarked_1.0,Embarked_2.0,Age_filtering_1,Age_filtering_2
0,-0.592481,0.432793,-0.473674,-0.502445,-0.510152,0.902587,0.737695,-0.307562,0.615838,-0.227921,0.379176
1,0.638789,0.432793,-0.473674,0.786845,-0.510152,-1.107926,-1.355574,-0.307562,-1.623803,-0.227921,0.379176
2,-0.284663,-0.474545,-0.473674,-0.488854,-0.510152,0.902587,-1.355574,-0.307562,0.615838,-0.227921,0.379176
3,0.407926,0.432793,-0.473674,0.420730,-0.510152,-1.107926,-1.355574,-0.307562,0.615838,-0.227921,0.379176
4,0.407926,-0.474545,-0.473674,-0.486337,-0.510152,0.902587,0.737695,-0.307562,0.615838,-0.227921,0.379176


In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(train, target)
predictions = model.predict(test)
scores = cross_val_score(model, train, target, cv=10)
scores.mean()

0.8316604244694131

# Didn't waste much time, just a simple fast model